In [22]:
import numpy as np
import pandas as pd
import gc

from scipy.fft import fft
from scipy.signal import blackman

from pandarallel import pandarallel
pandarallel.initialize()

In [2]:
def seed_everything(seed):
    np.random.seed(seed)
    # tf.random.set_seed(seed)
    # os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(71)

In [3]:
DEBUG = False

submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

# ここのファイルは元のtrain.csv, test.csvで問題ないです
if DEBUG:
    # train = pd.read_csv('../input/datasets/train_features1_debug.csv')
    # test = pd.read_csv('../input/datasets/test_features1.csv')
    train = pd.read_pickle('../input/datasets/train_features1_debug.pkl')
    test = pd.read_pickle('../input/datasets/test_features1.pkl')
else:
    # train = pd.read_csv('../input/datasets/train_features1.csv')
    # test = pd.read_csv('../input/datasets/test_features1.csv')
    train = pd.read_pickle('../input/datasets/train_features1.pkl')
    test = pd.read_pickle('../input/datasets/test_features1.pkl')

# FFT

In [19]:
def FFT(df):    
    N = 80
    T = 1.0 
    y = df['u_in'].to_numpy()

    w = blackman(N)
    
    yf = fft(y)
    ywf = fft(y*w)
    df['fft'] = np.log(yf)
    df['fft_w'] = np.log(ywf)

    yf_cos = fft(np.cos(np.deg2rad(y)))
    ywf_cos = fft(np.cos(np.deg2rad(y))*w)
    df['fft_cos'] = np.log(yf_cos)
    df['fft_cos_w'] = np.log(ywf_cos)

    del N, T, y, w, yf, ywf, yf_cos, ywf_cos
    gc.collect()

    return df

In [35]:
print('Processing train...')
train = train.groupby('breath_id').parallel_apply(FFT)

print('Make train file...')
train.to_pickle('../input/dataset/train_fft.pkl', index=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Processing train...


ipykernel_launcher:12: RuntimeWarning: divide by zero encountered in log
ipykernel_launcher:13: RuntimeWarning: divide by zero encountered in log
ipykernel_launcher:17: RuntimeWarning: divide by zero encountered in log
ipykernel_launcher:18: RuntimeWarning: divide by zero encountered in log


KeyboardInterrupt: 

Process ForkPoolWorker-119:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/shugo/Desktop/Competitions/Kaggle/Kaggle_Google-Brain-Ventilator-Pressure-Prediction/venv_gb/lib/python3.7/site-packages/pandarallel/pandarallel.py", line 64, in global_worker
    return _func(x)
  File "/Users/shugo/Desktop/Competitions/Kaggle/Kaggle_Google-Brain-Ventilator-Pressure-Prediction/venv_gb/lib/python

In [ ]:
print('Processing test...')
test = test.groupby('breath_id').parallel_apply(FFT)

print('Make test file...')
test.to_pickle('../input/dataset/test_fft.pkl', index=False)